In [1]:
import pandas as pd
import numpy as np
import pyreadr
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import linregress
import os

In [2]:
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 100)

In [20]:
data_predict = pd.read_pickle('pickles/data_predict_2009.pkl')
data_predict = data_predict.dropna()
display(data_predict)

,song_id,tag,release_date,release_month,release_year,listen_times,chroma_stft_norm,rmse_norm,spectral_centroid_norm,spectral_bandwidth_norm,rolloff_norm,zero_crossing_rate_norm,mfcc1_norm,mfcc2_norm,mfcc3_norm,mfcc4_norm,mfcc5_norm,mfcc6_norm,mfcc7_norm,mfcc8_norm,mfcc9_norm,mfcc10_norm,mfcc11_norm,mfcc12_norm,mfcc13_norm,...,mfcc15_norm,mfcc16_norm,mfcc17_norm,mfcc18_norm,mfcc19_norm,mfcc20_norm,tag_bin,treatment_date,treatment_year,gnb_pred,gnb_pred_acc,gnb_pred_bin,gnb_pred_bin_acc,knn_pred,knn_pred_acc,knn_pred_bin,knn_pred_bin_acc,rf_pred,rf_pred_acc,rf_pred_bin,rf_pred_bin_acc,xgb_pred,xgb_pred_acc,xgb_pred_bin,xgb_pred_bin_acc
1,xNgVaKafcd0,pop,2017-09-08,2017.666667,2017,952.0,0.313235,0.226911,0.325286,0.570343,0.463045,0.159881,0.802928,0.438278,0.463192,0.548091,0.388213,0.378023,0.495891,0.288580,0.488314,0.653325,0.411552,0.365625,0.359362,...,0.461608,0.443594,0.433783,0.280255,0.320974,0.607268,nonhiphop,False,0,folk,False,nonhiphop,True,pop,1,nonhiphop,1,folk,False,nonhiphop,True,3.0,False,nonhiphop,True
2,mSOdaq656ff,rock,2015-11-11,2015.833333,2015,601.0,0.553648,0.305978,0.255246,0.479821,0.377546,0.099607,0.824299,0.524473,0.405405,0.709515,0.526825,0.493619,0.486819,0.663650,0.652753,0.769993,0.717449,0.484404,0.558664,...,0.536892,0.555597,0.530681,0.281744,0.344796,0.510593,nonhiphop,True,1,rock,True,nonhiphop,True,rock,1,nonhiphop,1,rock,True,nonhiphop,True,1.0,False,nonhiphop,True
3,bqv5oMs3aac9,pop,2016-05-24,2016.333333,2016,425.0,0.317953,0.153784,0.248175,0.558457,0.349581,0.093165,0.757307,0.546043,0.536335,0.520767,0.543061,0.461001,0.461963,0.549711,0.498131,0.638340,0.569084,0.339578,0.534756,...,0.505196,0.531985,0.452319,0.275470,0.249064,0.486048,nonhiphop,True,1,folk,False,nonhiphop,True,pop,1,nonhiphop,1,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
4,xLBr1fbf8ac,pop,2015-06-27,2015.416667,2015,42450.0,0.295309,0.174921,0.247695,0.550785,0.362049,0.090020,0.749085,0.521823,0.549510,0.566503,0.522691,0.396086,0.530403,0.533256,0.584893,0.616224,0.567201,0.407748,0.466743,...,0.478182,0.539164,0.399455,0.224764,0.347304,0.591887,nonhiphop,True,1,folk,False,nonhiphop,True,pop,1,nonhiphop,1,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
5,bqwPhDt3af1d,pop,2016-04-14,2016.250000,2016,352.0,0.359844,0.161044,0.276050,0.628411,0.440350,0.079107,0.746165,0.476319,0.578945,0.551392,0.510643,0.440828,0.633674,0.502060,0.614375,0.721540,0.639329,0.453601,0.532991,...,0.473726,0.570167,0.437348,0.286538,0.284802,0.470208,nonhiphop,True,1,pop,True,nonhiphop,True,pop,1,nonhiphop,1,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69422,mQMTv46f559,pop,2016-03-30,2016.166667,2016,376342.0,0.422683,0.262165,0.391139,0.677416,0.601271,0.162169,0.836992,0.345847,0.477700,0.562698,0.542400,0.441040,0.513699,0.664571,0.622143,0.711451,0.635888,0.443183,0.518191,...,0.503307,0.617408,0.503275,0.324825,0.313520,0.537919,nonhiphop,True,1,pop,True,nonhiphop,True,pop,1,nonhiphop,1,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
69423,JCD4Da24686,folk,2017-10-18,2017.750000,2017,2881.0,0.268429,0.071585,0.177261,0.383710,0.233001,0.075891,0.642775,0.603658,0.539333,0.432852,0.449927,0.323510,0.469320,0.428199,0.379113,0.669085,0.536154,0.382162,0.492283,...,0.487365,0.467437,0.350374,0.284681,0.206697,0.372778,nonhiphop,False,0,pop,False,nonhiphop,True,pop,0,nonhiphop,1,pop,False,nonhiphop,True,1.0,False,nonhiphop,True
69424,mQPtrA69525,rock,2015-02-11,2015.083333,2015,908.0,0.454527,0.309575,0.271151,0.630601,0.463874,0.069429,0.808198,0.462401,0.557184,0.596576,0.579943,0.497058,0.509310,0.619674,0.595326,0.707396,0.692057,0.464133,0.559957,...,0.495488,0.629561,0.445785,0.262465,0.305179,0.534987,nonhiphop,True,1,rock,True,nonhiphop,True,pop,0,nonhiphop,1,pop,False,nonhiphop,True,1.0,False,nonhiphop,Tru

In [21]:
subg = pd.concat([pd.Series(data_predict.index), data_predict], axis = 1).rename(columns={0:'id'})
subg=subg.dropna()
display(subg)

,id,song_id,tag,release_date,release_month,release_year,listen_times,chroma_stft_norm,rmse_norm,spectral_centroid_norm,spectral_bandwidth_norm,rolloff_norm,zero_crossing_rate_norm,mfcc1_norm,mfcc2_norm,mfcc3_norm,mfcc4_norm,mfcc5_norm,mfcc6_norm,mfcc7_norm,mfcc8_norm,mfcc9_norm,mfcc10_norm,mfcc11_norm,mfcc12_norm,...,mfcc15_norm,mfcc16_norm,mfcc17_norm,mfcc18_norm,mfcc19_norm,mfcc20_norm,tag_bin,treatment_date,treatment_year,gnb_pred,gnb_pred_acc,gnb_pred_bin,gnb_pred_bin_acc,knn_pred,knn_pred_acc,knn_pred_bin,knn_pred_bin_acc,rf_pred,rf_pred_acc,rf_pred_bin,rf_pred_bin_acc,xgb_pred,xgb_pred_acc,xgb_pred_bin,xgb_pred_bin_acc
1,2.0,xNgVaKafcd0,pop,2017-09-08,2017.666667,2017.0,952.0,0.313235,0.226911,0.325286,0.570343,0.463045,0.159881,0.802928,0.438278,0.463192,0.548091,0.388213,0.378023,0.495891,0.288580,0.488314,0.653325,0.411552,0.365625,...,0.461608,0.443594,0.433783,0.280255,0.320974,0.607268,nonhiphop,False,0.0,folk,False,nonhiphop,True,pop,1.0,nonhiphop,1.0,folk,False,nonhiphop,True,3.0,False,nonhiphop,True
2,3.0,mSOdaq656ff,rock,2015-11-11,2015.833333,2015.0,601.0,0.553648,0.305978,0.255246,0.479821,0.377546,0.099607,0.824299,0.524473,0.405405,0.709515,0.526825,0.493619,0.486819,0.663650,0.652753,0.769993,0.717449,0.484404,...,0.536892,0.555597,0.530681,0.281744,0.344796,0.510593,nonhiphop,True,1.0,rock,True,nonhiphop,True,rock,1.0,nonhiphop,1.0,rock,True,nonhiphop,True,1.0,False,nonhiphop,True
3,4.0,bqv5oMs3aac9,pop,2016-05-24,2016.333333,2016.0,425.0,0.317953,0.153784,0.248175,0.558457,0.349581,0.093165,0.757307,0.546043,0.536335,0.520767,0.543061,0.461001,0.461963,0.549711,0.498131,0.638340,0.569084,0.339578,...,0.505196,0.531985,0.452319,0.275470,0.249064,0.486048,nonhiphop,True,1.0,folk,False,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
4,5.0,xLBr1fbf8ac,pop,2015-06-27,2015.416667,2015.0,42450.0,0.295309,0.174921,0.247695,0.550785,0.362049,0.090020,0.749085,0.521823,0.549510,0.566503,0.522691,0.396086,0.530403,0.533256,0.584893,0.616224,0.567201,0.407748,...,0.478182,0.539164,0.399455,0.224764,0.347304,0.591887,nonhiphop,True,1.0,folk,False,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
5,6.0,bqwPhDt3af1d,pop,2016-04-14,2016.250000,2016.0,352.0,0.359844,0.161044,0.276050,0.628411,0.440350,0.079107,0.746165,0.476319,0.578945,0.551392,0.510643,0.440828,0.633674,0.502060,0.614375,0.721540,0.639329,0.453601,...,0.473726,0.570167,0.437348,0.286538,0.284802,0.470208,nonhiphop,True,1.0,pop,True,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48591,69422.0,bf0Ijk82f435,folk,2016-02-01,2016.083333,2016.0,137.0,0.341621,0.289748,0.209546,0.464558,0.318685,0.077811,0.772771,0.584609,0.602656,0.606203,0.472807,0.425914,0.481122,0.573624,0.547234,0.694810,0.604919,0.407287,...,0.482108,0.497709,0.409901,0.273709,0.282566,0.420964,nonhiphop,True,1.0,pop,False,nonhiphop,True,pop,0.0,nonhiphop,1.0,folk,True,nonhiphop,True,1.0,False,nonhiphop,True
48592,69423.0,xLxgLOb2928,pop,2012-06-01,2012.416667,2012.0,8306.0,0.301727,0.172675,0.307115,0.607126,0.461280,0.128149,0.769921,0.480370,0.524757,0.487034,0.515817,0.384866,0.463026,0.592409,0.409282,0.667526,0.550570,0.358222,...,0.434522,0.570967,0.447933,0.294721,0.325187,0.603564,nonhiphop,True,1.0,pop,True,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True
48593,69424.0,8I4xIef658d,pop,2018-09-26,2018.666667,2018.0,1182.0,0.281489,0.085685,0.305004,0.605662,0.502154,0.102198,0.705823,0.414058,0.550184,0.596413,0.379706,0.395464,0.292855,0.461704,0.350694,0.505357,0.602169,0.271618,...,0.288137,0.429048,0.215878,0.181892,0.203569,0.410784,nonhiphop,False,0.0,pop,True,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,

In [22]:
subgenres= data_predict.groupby(['song_id','tag']).size().reset_index().pivot(index = 'song_id', columns = 'tag', values = 0)

In [23]:
display(subgenres)
print(data_predict.index)

tag,folk,hiphop,pop,rock
song_id,,,,
4jBabaec,NaN,NaN,1.0,NaN
4jC543be,NaN,NaN,1.0,NaN
4jL7d403,NaN,NaN,1.0,NaN
4jP8bb4b,NaN,NaN,1.0,NaN
4jS53fce,NaN,NaN,1.0,NaN
...,...,...,...,...
xOoghtdfa82,NaN,NaN,1.0,NaN
xOoghub575e,NaN,NaN,1.0,NaN
xOoghwbc25a,NaN,NaN,1.0,NaN


Index([    1,     2,     3,     4,     5,     6,     7,     8,     9,    11,
       ...
       69415, 69417, 69418, 69419, 69421, 69422, 69423, 69424, 69425, 69426],
      dtype='int64', length=48596)


In [24]:
data_predict_subgenres = pd.concat([data_predict, subgenres], axis = 1)
display(data_predict_subgenres)

,song_id,tag,release_date,release_month,release_year,listen_times,chroma_stft_norm,rmse_norm,spectral_centroid_norm,spectral_bandwidth_norm,rolloff_norm,zero_crossing_rate_norm,mfcc1_norm,mfcc2_norm,mfcc3_norm,mfcc4_norm,mfcc5_norm,mfcc6_norm,mfcc7_norm,mfcc8_norm,mfcc9_norm,mfcc10_norm,mfcc11_norm,mfcc12_norm,mfcc13_norm,...,mfcc19_norm,mfcc20_norm,tag_bin,treatment_date,treatment_year,gnb_pred,gnb_pred_acc,gnb_pred_bin,gnb_pred_bin_acc,knn_pred,knn_pred_acc,knn_pred_bin,knn_pred_bin_acc,rf_pred,rf_pred_acc,rf_pred_bin,rf_pred_bin_acc,xgb_pred,xgb_pred_acc,xgb_pred_bin,xgb_pred_bin_acc,folk,hiphop,pop,rock
1,xNgVaKafcd0,pop,2017-09-08,2017.666667,2017.0,952.0,0.313235,0.226911,0.325286,0.570343,0.463045,0.159881,0.802928,0.438278,0.463192,0.548091,0.388213,0.378023,0.495891,0.288580,0.488314,0.653325,0.411552,0.365625,0.359362,...,0.320974,0.607268,nonhiphop,False,0.0,folk,False,nonhiphop,True,pop,1.0,nonhiphop,1.0,folk,False,nonhiphop,True,3.0,False,nonhiphop,True,NaN,NaN,NaN,NaN
2,mSOdaq656ff,rock,2015-11-11,2015.833333,2015.0,601.0,0.553648,0.305978,0.255246,0.479821,0.377546,0.099607,0.824299,0.524473,0.405405,0.709515,0.526825,0.493619,0.486819,0.663650,0.652753,0.769993,0.717449,0.484404,0.558664,...,0.344796,0.510593,nonhiphop,True,1.0,rock,True,nonhiphop,True,rock,1.0,nonhiphop,1.0,rock,True,nonhiphop,True,1.0,False,nonhiphop,True,NaN,NaN,NaN,NaN
3,bqv5oMs3aac9,pop,2016-05-24,2016.333333,2016.0,425.0,0.317953,0.153784,0.248175,0.558457,0.349581,0.093165,0.757307,0.546043,0.536335,0.520767,0.543061,0.461001,0.461963,0.549711,0.498131,0.638340,0.569084,0.339578,0.534756,...,0.249064,0.486048,nonhiphop,True,1.0,folk,False,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,NaN,NaN,NaN,NaN
4,xLBr1fbf8ac,pop,2015-06-27,2015.416667,2015.0,42450.0,0.295309,0.174921,0.247695,0.550785,0.362049,0.090020,0.749085,0.521823,0.549510,0.566503,0.522691,0.396086,0.530403,0.533256,0.584893,0.616224,0.567201,0.407748,0.466743,...,0.347304,0.591887,nonhiphop,True,1.0,folk,False,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,NaN,NaN,NaN,NaN
5,bqwPhDt3af1d,pop,2016-04-14,2016.250000,2016.0,352.0,0.359844,0.161044,0.276050,0.628411,0.440350,0.079107,0.746165,0.476319,0.578945,0.551392,0.510643,0.440828,0.633674,0.502060,0.614375,0.721540,0.639329,0.453601,0.532991,...,0.284802,0.470208,nonhiphop,True,1.0,pop,True,nonhiphop,True,pop,1.0,nonhiphop,1.0,pop,True,nonhiphop,True,1.0,False,nonhiphop,True,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
xOoghtdfa82,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
xOoghub575e,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
xOoghwbc25a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
xOoghya0b0f,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
